Bevor Sie dieses Problem abgeben, vergewissern Sie sich, dass alles wie erwartet funktioniert. **Neustarten Sie den Kernel** und führen Sie dann alle Zellen einmal mit **Run All** aus.

Gehen Sie sicher, dass Sie alle Stellen ausgefüllt haben in denen steht `YOUR CODE HERE`.

Alle notwendigen Funktionen werden bereits importiert. Importieren Sie keine weiteren Pakete, dass schließt auch `math`ein! Importe können zu Punktabzug führen.

##### Vorrechen Aufgabe
Wir wollen die schnellsten $k$ Laufzeiten aus einem Datenstrom von Laufzeiten bestimmen. Dabei möchten wir nur O(k) Speicherplatz verbrauchen und unsere Operationen sollen effizient sein.
Überlege, ob ein sortiertes Array oder eine sortierte verkettete Liste (Linked List) geeignet ist und wie die Laufzeiten für die Operationen (Suchen, Einfügen/Löschen, Ausgeben) sind.
Vergleiche dies mit einem Heap. Bedenke auch, ob ein Min-Heap oder ein Max-Heap besser geeignet ist.
Implementiere die effizienteste Datenstruktur.

In [ ]:
import bisect # Für Sortiertes Array
import heapq # Für Heap

class TopK():
    # YOUR CODE HERE
    raise NotImplementedError("Hier fehlt deine Lösung.")

In [ ]:
# Diese Aufgabe gibt keine Punkte
datenstrom = [5,9,2,7,8,3,4,1]

top_k = TopK(3)

for runtime in datenstrom:
    top_k.insert(runtime)
    print(top_k)

print(top_k)


Wir haben zuletzt Suchbäume und Hashing kennengelernt. Als abstrakte Datenstrukturen bieten beide die Operationen: Finden, Einfügen und Löschen. Ihre Implementierungen und Laufzeiten unterscheiden sich jedoch deutlich. Beginnen wir damit, AB-Bäume und linear sondiertes Hashing zu implementieren. Danach wollen wir für zwei Problemstellungen auswerten, welche Datenstruktur geeigneter ist.

### Aufgabe 1
Beginnen wir mit der Implementierung von AB-Bäumen. Ich habe euch bereits Klassen vorgegeben und locate implementiert. Als Nächstes wollen wir die Funktionalität für das Einfügen implementieren. Nutzt dazu Seite 193 aus dem Buch “Algorithmen und Datenstrukturen” von Mehlhorn und Sanders als Hilfe.

In [ ]:
from typing import Generator, Self, Any, TypeVar, Tuple

TEST_INVAR = True

D = TypeVar('Data')

class Item:
    def __init__(self, data : D):
        self.data = data
        self.next : Self = self
        self.prev : Self = self

    def __repr__(self) -> str:
        return str(self.data)

    def invarianten(self) -> bool:
        if not TEST_INVAR:
            return True
        assert self.next.prev == self, f"next.prev {self.next.prev} ist nicht {self}"
        assert self.prev.next == self, f"prev.next {self.prev.next} ist nicht {self}"
        return True

class DLList:
    def __init__(self, datas : Tuple[D, ...]):
        self.length : int = 0
        self.head : Item = Item(None)
        assert self.invarianten()
        for data in datas:
            self.insertAfter(self.head.prev, data)
        assert self.invarianten()

    def __repr__(self) -> str:
        return "[" + ", ".join(str(item) for item in self) + "]"

    def insertAfter(self, item : Item, data : D) -> Item:
        new_item = Item(data)
        fort = item.next
        item.next = new_item
        new_item.next = fort
        fort.prev = new_item
        new_item.prev = item
        self.length += 1
        assert self.invarianten()
        return new_item

    def insertBefore(self, item : Item, data : D) -> Item:
        return self.insertAfter(item.prev, data)

    def __len__(self) -> int:
        return self.length

    def __iter__(self) -> Generator[Item, Any, Any]:
        item = self.head.next
        while item != self.head:
            yield item
            item = item.next

    def invarianten(self) -> bool:
        if not TEST_INVAR:
            return True
        assert self.head.prev.next == self.head, f"head.prev.next ist nicht head"
        assert self.head.next.prev == self.head, f"head.next.prev ist nicht head"
        assert self.length == sum(1 for _ in self), f"length {self.length} passt nicht zu Anzahl Elemente {sum(1 for _ in self)}"
        for item in self:
            assert item.invarianten(), f"Das Element {item} ist ungültig"
        return True


In [ ]:
from typing import Generator, Self, Any, TypeVar, Tuple
import bisect

K = TypeVar('Key')
 

def data_to_key(data : D) -> K:
    if hasattr(data, 'key'):
        return data.key
    else:
        return data


class ABItem:
    def __init__(self, a : int, b : int, splitters : Tuple[K, ...], children : Tuple[Self | Item, ...]):
        self.a = a
        self.b = b
        self.d = len(children)
        self.splitter = splitters
        self.children = children

    def locateLocally(self, key : K) -> int:
        return bisect.bisect_left(self.splitter, key, 0, self.d-1) # Binary Search
        return min([i for i in range(self.d-1) if key <= self.splitter[i]], default=self.d-1) # Linear Search

    def locateRec(self, key: K, height: int) -> Item:
        index = self.locateLocally(key)
        if height == 1:
            if data_to_key(self.children[index].data) >= key:
                return self.children[index]
            else:
                return self.children[index].next
        else:
            return self.children[index].locateRec(key, height - 1)

    def insertRec(self, data : D, height: int, liste: DLList) -> tuple[K | None, Item | None]:
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

    def __iter__(self) -> Generator[Self | Item, Any, Any]:
        for child in self.children:
            if hasattr(child, "children"):
                yield from child
            else:
                yield child

    def __str__(self):
        children_strings = [str(child).split("\n") for child in self.children]
        splitter_strings = [str(splitter) for splitter in self.splitter]
        # Generate Top line
        top = ""
        for clines, sstr in zip(children_strings, splitter_strings):
            width = max(len(line) for line in clines)
            top += "_"*(width+1-len(sstr)) + sstr
            # Add Spacer between Subtrees
            for idx, cline in enumerate(clines):
                clines[idx] = cline + "|"
        else:
            clines = children_strings[-1]
            width = max(len(line) for line in clines)
            top += "_"*width
        # Generate Rest
        bottom = ""
        for depth in zip(*children_strings):
            row = "".join(depth)
            bottom += row + "\n"
        bottom = bottom[:-1] #Remove the last newline

        return top + "\n" + bottom

    def invarianten(self, start : bool=False) -> bool:
        if not TEST_INVAR:
            return True
        assert self.a >= 2, f"a {self.a} muss mindestens 2 sein"
        assert self.b >= 2 * self.a - 1, f"b {self.b} muss mindestens 2a - 1 {2 * self.a - 1} sein"
        assert len(self.splitter) == len(self.children) - 1, f"splitter {self.splitter} und children {self.children} passen nicht zusammen"
        if not start:
            assert self.a <= len(self.children), f"{self} hat zu wenige Kinder: a:{self.a}, len(self):{len(self.children)}"
        assert len(self.children) <= self.b, f"{self} hat zu viele Kinder"
        assert self.d == len(self.children), f"d passt nicht zu children"
        for i in range(1, self.d):
            if isinstance(self.children[i], Item):
                assert self.splitter[i-1] < data_to_key(self.children[i].data), f"Kind {self.children[i]} ist kleinergleich als vorheriger splitter {self.splitter[i-1]}"
            else:
                for leaf in self.children[i]:
                    assert self.splitter[i-1] < data_to_key(leaf.data), f"Kind {leaf} ist kleinergleich als vorheriger splitter {self.splitter[i-1]}"
        for i in range(self.d-1):
            if isinstance(self.children[i], Item):
                assert data_to_key(self.children[i].data) <= self.splitter[i], f"Kind {self.children[i]} ist größer als nächster splitter {self.splitter[i]}"
            else:
                for leaf in self.children[i]:
                    assert data_to_key(leaf.data) <= self.splitter[i], f"Kind {leaf} ist größer als nächster splitter self.splitter[i]"
        for child in self.children:
            assert child.invarianten(), "Das Kind {child} ist ungültig."
        return True

class ABTree:
    def __init__(self, a: int, b: int):
        self.a = a
        self.b = b
        self.liste = DLList([float("inf")])
        self.root = ABItem(2, self.b, [], [self.liste.head.next])
        self.height = 1
        assert self.invarianten(), "nach init ist der Baum ungültig"

    def locate(self, key : K) -> Item:
        return self.root.locateRec(key, self.height)

    def insert(self, data : D):
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")
        assert self.invarianten(), f"nach insert von Data {data} ist der Baum ungültig"

    def __str__(self) -> str:
        return str(self.root)

    def invarianten(self):
        if not TEST_INVAR:
            return True
        assert self.a >= 2, f"a {self.a} muss mindestens 2 sein"
        assert self.b >= 2 * self.a - 1, f"b {self.b} muss mindestens 2a - 1 {2 * self.a - 1} sein"
        assert self.height >= 1, f"die Höhe {self.height} muss positiv sein"
        if len(self.liste) <= self.b:
            assert self.root.invarianten(start=True), "root ist nicht gültig"
        else:
            assert self.root.invarianten(), "root ist nicht gültig"
        return True

In [ ]:
# Testet, ob bei ihrer Implementierung ohne Änderung alles Funktioniert.
TEST_INVAR = True

import random
random.seed(42)

for _ in range(10):
    a = random.randint(2, 5)
    b = random.randint(a*2-1, 4*a)
    items = set([float('inf')])
    tree = ABTree(a, b)
    for _ in range(100):
        rkey = random.randint(0, 99)
        items.add(rkey)
        tree.insert(rkey)
        assert set([ele.data for ele in tree.liste]) == items, f"Im ABBaum sind die Elemente {tree.liste}, es sollten aber die Elemente {list(items)} enthalten sein."
        assert TEST_INVAR
        assert tree.invarianten()
TEST_INVAR = False # Deaktiviert die Invarianten, damit die Implementierung später schneller läuft.
print('Alle Tests erfolgreich.')

### Aufgabe 2
Nun brauchen wir noch eine Implementierung einer Hashtabelle zum Vergleich. Implementieren Sie dafür die Klasse `LinearHashing`. Sie soll ein Array der Länge $k$ verwenden, und als Hashfunktion verwenden wir `%k`. `LinearHashing` soll die Lücken füllen, die beim Löschen entstehen. Hier können Sie sich an den Folien der Vorlesung orientieren.

In [ ]:
def data_to_key(data : D) -> K:
    if hasattr(data, 'key'):
        return data.key
    else:
        return data

class LinearHashing:

    def __init__(self, l : int) -> None:
        self.l = l
        self.liste = [None] * l

    def insert(self, data : D) -> bool: # Gibt True zurück, wenn der Wert eingefügt wurde, und False, falls es keine freie Stelle mehr gab.
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

    def delete(self, data : D) -> bool: # Gibt True zurück, wenn der Wert gefunden und gelöscht wurde, ansonsten False
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

    def fill_empty(self, index : int) -> None:
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")


    def locate(self, key : K) -> D:
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")


    def search(self, key : K) -> bool:
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

In [ ]:
import random
random.seed(42)

for i in range(1,4):
    ref_set = set()
    lh = LinearHashing(10**i)
    actions = random.choices(["insert", "delete", "search"], k=15**i)
    keys = random.choices(range(10_000), k=15**i)
    for action, key in zip(actions, keys):
        assert len(ref_set) == sum([1 for val in lh.liste if val is not None]), f"len(ref_set)={len(ref_set)}, sum([1 for val in vh.liste if val is not None])={sum([1 for val in lh.liste if val is not None])}"
        if action == "insert":
            if len(ref_set) >= 10**i:
                continue
            lh.insert(key)
            ref_set.add(key)
        elif action == "delete":
            index = lh.liste.index(key) if key in ref_set else None
            erg = lh.delete(key)
            assert erg == (key in ref_set), f"erg={erg}, key={key}, ref_set={ref_set}, vh.liste={lh.liste}"
            ref_set.discard(key)
        elif action == "search":
            assert lh.search(key) == (key in ref_set), f"key={key}, ref_set={ref_set}, vh.liste={lh.liste}"
        assert len(ref_set) == sum([1 for val in lh.liste if val is not None]), f"len(ref_set)={len(ref_set)}, sum([1 for val in vh.liste if val is not None])={sum([1 for val in lh.liste if val is not None])}"
        for idx, val in enumerate(lh.liste):
            if val == None:
                continue
            if idx > val % lh.l:
                for j in range(val % lh.l, idx):
                    assert lh.liste[j] != None, f"j={j}, k={val % lh.l}, idx={idx}, action={action}, key={key}, lh.liste={lh.liste}"
            elif idx < val % lh.l:
                for j in range(val % lh.l, len(lh.liste)):
                    assert lh.liste[j] != None, f"j={j}, k={val % lh.l}, idx={idx}, action={action}, key={key}, lh.liste={lh.liste}"
                for j in range(0, idx):
                    assert lh.liste[j] != None, f"j={j}, k={val % lh.l}, idx={idx}, action={action}, key={key}, lh.liste={lh.liste}"


print('Alle Tests erfolgreich.')

### Aufgabe 3
Wir möchten ein neues ML-Modell launchen und benötigen einen Onlineservice, über den wir es anbieten können. Dafür müssen wir viele neue Nutzer registrieren können, insbesondere aber mittels eines gehashten Passwortes verifizieren. Damit der Launch ein Erfolg wird, wollen wir sicherstellen, dass kein Nutzer unnötig lange auf eine Antwort warten muss. Wir können jedoch nicht abschätzen, wie viele neue Nutzer wir erwarten können. Welche Datenstruktur passt für dieses Szenario? Implementiere dafür UserDB und nutze deine Datenstruktur.

In [ ]:
class User:
    
    def __init__(self, pass_hash):
        self.pass_hash = pass_hash
        self.key = pass_hash

class UserDB:

    def __init__(self):
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

    def add(self, pass_hash) -> None:
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

    def verify(self, pass_hash) -> bool:
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

In [ ]:
from time import perf_counter

TEST_INVAR = False

def timer(f):
    def wrapper(*args, **kwargs):
        start = perf_counter()
        erg = f(*args, **kwargs)
        end = perf_counter()
        print(f"{f.__qualname__} hat {end-start} Sekunden gebraucht")
        return erg
    return wrapper

userdb = UserDB()
for user in range(10000):
    userdb.add(hash(str(user)))

@timer
def last_add():
    userdb.add(hash(str(-1)))

last_add()

for user in range(1, 20000, 2):
    userdb.verify(hash(str(user)))

@timer
def last_verify():
    return userdb.verify(hash(str(1)))

@timer
def last_verify2():
    return userdb.verify(hash(str(-2)))

assert last_verify()
assert not last_verify2()
print('Alle Tests erfolgreich.')


### Aufgabe 4
Nicht jedem gefällt, dass wir das beste ML-Modell für wenig Geld zur Verfügung stellen. Deshalb wollen wir einen DDOS-Filter erstellen. Dieser soll mittels der IP-Adresse prüfen, ob wir bereits mehr als 1000 Anfragen von derselben IP erhalten haben. Ab dann soll der Filter die Anfragen nicht mehr an den Loginservice weiterleiten. Da dieser Schutz nur eine Entlastung, aber nicht essentiell ist, kann er für Wartungsarbeiten auch mal ausfallen. Die Anfragen werden dann direkt an den Login weitergeleitet. Allerdings erwarten wir, dass der Service die Responsezeit kaum beeinflusst. Auch können wir vorab nicht sagen, von wie vielen IP-Adressen wir Anfragen bekommen. Welche Datenstruktur wäre für dieses Szenario geeignet? Implementiere dafür IPDB und nutze eine deiner Datenstrukturen. (Leider haben wir keine Zeit für Multiprocessing/Threading, wir können also das Ausfallen eines Services, das Weiterleiten der Anfrage und parallele Wartungsarbeiten nicht umsetzen. Führe deine Wartungsarbeiten einfach durch und beantworte dann die Anfrage.)

In [ ]:
class IPBanner:

    def __init__(self, ip):
        self.ip = ip
        self.key = ip
        self.counter = 0

    @property
    def valid(self):
        self.counter += 1
        if self.counter >= 1000:
            return False
        return True

class IPDB:

    def __init__(self):
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

    def verify(self, ip):
        # YOUR CODE HERE
        raise NotImplementedError("Hier fehlt deine Lösung.")

In [ ]:
from time import perf_counter
import random
random.seed(42)

TEST_INVAR = False

def timer(f):
    def wrapper(*args, **kwargs):
        start = perf_counter()
        erg = f(*args, **kwargs)
        end = perf_counter()
        print(f"{f.__qualname__} hat {end-start} Sekunden gebraucht")
        return erg
    return wrapper

@timer
def test_not_valid():
    return not ipdb.verify(hash(str(1)))

@timer
def test_fill_up():
    for _ in range(999):
        assert ipdb.verify(hash(str(-1)))

    return not ipdb.verify(hash(str(-1)))

ipdb = IPDB()
for k in range(999):
    if random.randint(0, 1):
        ipdb.verify(hash(str(1)))
    else:
        ipdb.verify(hash(str(k)))
    

assert ipdb.verify(hash(str(1)))

for k in range(1999):
    if random.randint(0, 1):
        ipdb.verify(hash(str(1)))
    else:
        ipdb.verify(hash(str(k)))
    
assert test_not_valid()

assert test_fill_up()
print('Alle Tests erfolgreich.')

Hier noch einmal die Aufgaben 3 und 4 in weniger angewandter Sprache: Beide Datenbanken müssen dynamisch sein, aber UserDB soll für jede Operation eine effiziente Laufzeit garantieren, während IPDB dies nur für möglichst viele Operationen muss. Bei IPDB können wir also in Einzelfällen eine längere Operation, wie das Neuerstellen der Datenstruktur, erlauben. Dafür können wir dann aber eine noch bessere Laufzeit für die Verifikation erwarten.